In [2]:
import struct
import numpy as np
import intvalpy as ip
import numpy as np
import readfiles

In [3]:
filepath_y = "0.225_lvl_side_a_fast_data.bin"
filepath_x = "-0.205_lvl_side_a_fast_data.bin"

In [4]:
rad = 1/(2**14)

X_data_origin = readfiles.read_files(filepath_x)
Y_data_origin = readfiles.read_files(filepath_y)


In [8]:
def get_avg(data):
    avg = [[0]*8]*1024
    for i in range(len(data)): # 100
        avg = np.add(avg, data[i])
    return np.divide(avg, len(data))

In [9]:
x_avg = get_avg(X_data_origin)
y_avg = get_avg(Y_data_origin)

In [12]:
import matplotlib.pyplot as plt

In [37]:
def scalar_to_interval(x, rad):
    return ip.Interval(x-rad, x+rad)

In [55]:
X = np.vectorize(scalar_to_interval)(x_avg, rad).flatten()
Y = np.vectorize(scalar_to_interval)(y_avg, rad).flatten()


In [53]:
# bounds
a_min = float(np.min(Y).a) - float(np.max(X).b)
a_max = float(np.max(Y).b) - float(np.min(X).a)


t_min = float(np.min(Y).a) / float(np.max(X).b) 
t_max = float(np.max(Y).b) / float(np.min(X).a)

In [78]:
def jaccart_interval(x, y):
    
    a1, b1 = float(x.a), float(x.b)
    a2, b2 = float(y.a), float(y.b)   
    return (min(b1, b2) - max(a1, a2)) / (max(b1, b2) - min(a1, a2))

def jaccart_intervals(X):
    lowers = [float(x.a) for x in X]
    uppers = [float(x.b) for x in X]
    return (min(uppers) - max(lowers)) / (max(uppers) - min(lowers))

def jaccart_sets(X, Y):
    coeffs = [jaccart_interval(x, y) for x, y in zip(X, Y)]
    return np.array(coeffs)

def coeff_Jaccart(X, Y):    
    if isinstance(X, ip.ClassicalArithmetic) and isinstance(Y, ip.ClassicalArithmetic):
        return jaccart_interval(X, Y)
    return jaccart_sets(X, Y)



In [99]:
def mean_jaccart_a(a):
    return np.mean(coeff_Jaccart(X+a, Y))

def mean_jaccart_t(t):
    return np.mean(coeff_Jaccart(X*t, Y))


def mean_jaccart_a_Kreynovich(a):
    return np.mean(coeff_Jaccart(Kreynovich(X+a), Kreynovich(Y)))

def mean_jaccart_t_Kreynovich(t):
    return np.mean(coeff_Jaccart(Kreynovich(X*t), Kreynovich(Y)))

In [98]:
def gold_search(func, a, b, eps=1e-4):
    
    phi = (3 - np.sqrt(5))/2
    
    x1 = a + phi*(b-a)
    x2 = b - phi*(b-a)
    
    f1 = func(x1) 
    f2 = func(x2)
    
    while abs(b - a) > eps:
        if f1 < f2:
            a = x1
            x1 = x2
            f1 = f2
            x2 = b - phi*(b-a)
            f2 = func(x2)
        else:
            b = x2
            x2 = x1
            f2 = f1
            x1 = a + phi*(b-a)
            f1 = func(x1)
            
    return (a+ b)/2

In [95]:
gold_search(mean_jaccart_a, a_min, a_max, eps=1e-4)

0.34072124626717126

In [96]:
gold_search(mean_jaccart_t, t_min, t_max, eps=1e-4)

-1.0371997472459227

In [ ]:
def Kreynovich(arr):
    
    left_bounds = [float(el.a) for el in x]
    right_bounds = [float(el.b) for el in x]
    
    return ip.Interval([float(np.median(lowers)), float(np.median(uppers))])